In [ ]:
import pandas as pd
import numpy as np
import relevanceeval

# Goal
This jupyter notebook serves as an area to test how to automatically test the results of our trained model to the expert annotations. It serves as a testing ground. 

In [ ]:
# Loading in the queries csv file.
queries = pd.read_csv("./Dataset/Testing/queries.csv")
queries

In [ ]:
# Load in the annotationStore csv file.
results = pd.read_csv("./Dataset/Testing/annotationStore.csv")
results

In [ ]:
# Utilizing the relevanceeval.py
relevance_scores = relevanceeval.load_relevances("./Dataset/Testing/annotationStore.csv")

In [ ]:
# Loading in a prediction file generated by me.
predictions = relevanceeval.load_predictions("./csv_output/baseline_50k.csv")

In [ ]:
# seing the languages
languages_predicted = sorted(set(predictions.keys()))
languages_predicted

In [ ]:
print('% of URLs in predictions that exist in the annotation dataset:')
for language in languages_predicted:
    print(f'\t{language}: {relevanceeval.coverage_per_language(predictions[language], relevance_scores[language])*100:.2f}%')

print('% of URLs in predictions that exist in the annotation dataset (avg relevance > 0):')
for language in languages_predicted:
    print(f'\t{language}: {relevanceeval.coverage_per_language(predictions[language], relevance_scores[language], with_positive_relevance=True) * 100:.2f}%')

print('NDCG:')
for language in languages_predicted:
    print(f'\t{language}: {relevanceeval.ndcg(predictions[language], relevance_scores[language]):.3f}')

In [ ]:
# Results for baseline 20k NO FILTERING ANSWERS
# % of URLs in predictions that exist in the annotation dataset:
# 	go: 0.00%
# 	java: 0.25%
# 	javascript: 0.00%
# 	php: 0.32%
# 	python: 0.20%
# 	ruby: 0.64%
# % of URLs in predictions that exist in the annotation dataset (avg relevance > 0):
# 	go: 0.00%
# 	java: 0.23%
# 	javascript: 0.00%
# 	php: 0.47%
# 	python: 0.23%
# 	ruby: 0.00%
# NDCG:
# 	go: 0.000
# 	java: 0.006
# 	javascript: 0.000
# 	php: 0.001
# 	python: 0.005
# 	ruby: 0.000

In [ ]:
#Newer working area

In [ ]:
annotationStore = pd.read_csv("./Dataset/Testing/annotationStore.csv")
b_50k = pd.read_csv("./csv_output/baseline_50k_2.csv")

In [ ]:
# b_50k
sum(pd.merge(b_50k, annotationStore, how="left", left_on="url", right_on="GitHubUrl")["GitHubUrl"].isnull() == False)

In [ ]:
annotationStore

In [ ]:
# annotationStore.drop_duplicates("GitHubUrl", keep="first").to_csv("./Dataset/Testing/annotationStore_UNIQUE.csv")

In [ ]:
relevance_annotations = pd.read_csv("./Dataset/Testing/annotationStore_UNIQUE.csv")
per_query_language = relevance_annotations.pivot_table(
    index=['Query', 'Language', 'GitHubUrl'], values='Relevance', aggfunc=np.mean)

In [ ]:
# per_query_language
relevance_annotations["Relevance"].value_counts()

In [ ]:
b_20k_test = pd.read_csv("./csv_output/baseline_20000k.csv")

In [ ]:
b_20k_test["relevance"] = [0 for i in range(len(b_20k_test))]

In [ ]:
annotationStore = pd.read_csv("./Dataset/Testing/annotationStore_UNIQUE.csv")

In [ ]:
b_20k_test

In [ ]:
res_annot_merged = pd.merge(b_50k, annotationStore, how="left", left_on="url", right_on="GitHubUrl")
ans_in_res_DF = res_annot_merged[res_annot_merged["GitHubUrl"].isnull() == False]

In [ ]:
res_annot_merged["Relevance"].fillna(0, inplace=True)#[res_annot_merged["url"].isnull() == False]

In [ ]:
res_annot_merged["Relevance"].value_counts()

In [ ]:
queries_with_annotations = list(ans_in_res_DF["query"].unique())

In [ ]:
ndcgs = []
for qwa in queries_with_annotations:
    df_query = res_annot_merged[res_annot_merged["query"] == qwa]
    # print(len(df_query))
    # print(df_query["Relevance"].to_list())
    rel_lst = df_query["Relevance"].to_list()

    dcg = 0
    for i, val in enumerate(rel_lst):
        dcg += (val) / (np.log2(i + 2))

    # print(dcg)
    idcg = 0
    for i, val in enumerate(sorted(rel_lst, reverse=True)):
        idcg += (val) / (np.log2(i + 2))
    # print(idcg)
    ndcgs.append(dcg / idcg)
print(np.mean(ndcgs))

In [ ]:
res_annot_merged

In [ ]:
res_annot_merged = pd.merge(b_50k, annotationStore, how="left", left_on="url", right_on="GitHubUrl")

ans_in_res_DF = res_annot_merged[res_annot_merged["GitHubUrl"].isnull() == False]#[["Language","Query", "GitHubUrl", "Relevance", "Notes"]]


In [ ]:
ans_in_res_DF.to_csv("./csv_output/rel_score_combined.csv")
ans_in_res_DF

In [ ]:
relevance_scores = relevanceeval.load_relevances("./csv_output/rel_score_combined.csv")
predictions = relevanceeval.load_predictions("./csv_output/baseline_50k.csv")
languages_predicted = sorted(set(predictions.keys()))
languages_rs = sorted(set(relevance_scores.keys()))

In [ ]:
predictions
relevance_scores

In [ ]:
print('% of URLs in predictions that exist in the annotation dataset:')
for language in languages_predicted:
    if language in languages_rs:
        print(f'\t{language}: {relevanceeval.coverage_per_language(predictions[language], relevance_scores[language])*100:.2f}%')

print('% of URLs in predictions that exist in the annotation dataset (avg relevance > 0):')
for language in languages_predicted:
    if language in languages_rs:
        print(f'\t{language}: {relevanceeval.coverage_per_language(predictions[language], relevance_scores[language], with_positive_relevance=True) * 100:.2f}%')

print('NDCG:')
for language in languages_predicted:
    if language in languages_rs:
        print(f'\t{language}: {relevanceeval.ndcg(predictions[language], relevance_scores[language]):.3f}')